## Zadanie 1 (2 pkt.)

Napisz funkcję rozstrzygającą czy dana lista jest palindromem, ale taką, która wykonuje co najwyżej ⌊n/2⌋ wywołań rekurencyjnych, gdzie n jest nieznaną długością listy (nie potrzeba i nie wolno jej liczyć). Funkcja nie powinna również alokować nowych list.

In [25]:
let is_palindrome l =
    let rec aux = function
        | (l,[]) -> (true,l)
        | (l,[x]) -> (true, List.tl l)
        | (x::xs, y::y'::ys) -> let p = aux (xs,ys) in
            let isp = fst p and list = snd p in
                if isp == false
                then (false,[])
                else if List.hd list == x 
                    then (true, List.tl list)
                    else (false, [])
    in
        fst @@ aux (l,l)

File "[25]", line 2, characters 18-387:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a value that is not matched:
([], _::_::_)


val is_palindrome : 'a list -> bool = <fun>


In [27]:
is_palindrome [];;
is_palindrome [1];;
is_palindrome [1;2];;
is_palindrome [1;1];;
is_palindrome [1;2;1];;
is_palindrome [1;2;2];;

- : bool = true


- : bool = true


- : bool = false


- : bool = true


- : bool = true


- : bool = false


## Zadanie 2 (2 pkt.)

Rozważmy typ danych dla drzew binarnych, zdefiniowany następująco:

`type 'a btree = Leaf | Node of 'a btree * 'a * 'a btree`

Mówimy, że drzewo jest zbalansowane, jeśli dla każdego węzła v liczby węzłów w lewym i prawym poddrzewie zakorzenionym w v różnią się co najwyżej o 1.

1. Napisz efektywną funkcję sprawdzającą czy dane drzewo jest zbalansowane.
2. Napisz funkcję, która dla zadanej listy etykiet tworzy zbalansowane drzewo, dla którego tę listę można otrzymać przechodząc je w porządku preorder.

In [5]:
type 'a btree = Leaf | Node of 'a btree * 'a * 'a btree

let is_tree_balanced t = 
    let rec aux = function
        | Leaf        -> (true, 1)
        | Node(l,_,r) -> let a = aux l and b = aux r in
            if abs (snd a - snd b) > 1
            then (false,  max (snd a) (snd b))
            else (fst a && fst b, 1 + max (snd a) (snd b)) in
    fst @@ aux t

let rec tree_to_list_VLR = function 
    | Leaf -> []
    | Node(left,e,right) -> e :: ( (tree_to_list_VLR left) @ (tree_to_list_VLR right) )

let halves l = 
    let rec aux acc = function
        | (r, []) -> (List.rev acc,r)
        | (r, [_]) -> (List.rev acc,r)
        | (a::r, _::_::q) -> aux (a::acc) (r,q)
    in
        aux [] (l,l)

let rec list_to_tree = function
    | [] -> Leaf
    | [x] -> Node(Leaf,x,Leaf)
    | x::xs -> let p = halves xs in
        let left = fst p and right = snd p in
        Node(list_to_tree left,x,list_to_tree right)

type 'a btree = Leaf | Node of 'a btree * 'a * 'a btree


val is_tree_balanced : 'a btree -> bool = <fun>


val tree_to_list_VLR : 'a btree -> 'a list = <fun>


File "[5]", line 17, characters 22-155:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a value that is not matched:
([], _::_::_)


val halves : 'a list -> 'a list * 'a list = <fun>


val list_to_tree : 'a list -> 'a btree = <fun>


In [10]:
let testList = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21];;
let tree = list_to_tree testList;;

is_tree_balanced tree;;
tree_to_list_VLR tree

val testList : int list =
  [1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21]


val tree : int btree =
  Node
   (Node
     (Node (Node (Leaf, 4, Leaf), 3, Node (Leaf, 5, Node (Leaf, 6, Leaf))),
     2,
     Node (Node (Leaf, 8, Node (Leaf, 9, Leaf)), 7,
      Node (Leaf, 10, Node (Leaf, 11, Leaf)))),
   1,
   Node
    (Node (Node (Leaf, 14, Leaf), 13, Node (Leaf, 15, Node (Leaf, 16, Leaf))),
    12,
    Node (Node (Leaf, 18, Node (Leaf, 19, Leaf)), 17,
     Node (Leaf, 20, Node (Leaf, 21, Leaf)))))


- : bool = true


- : int list =
[1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21]


## Zadanie 3 (2 pkt.)

Napisz możliwie efektywne funkcje przejścia wszerz oraz przejścia w głąb (preorder) dla obu reprezentacji drzew wielokierunkowych podanych na wykładzie (zadanie 5 z listy kontrolnej do wykładu 4).

In [12]:
type 'a mtree = MNode of 'a * 'a forest
and 'a forest = EmptyForest | Forest of 'a mtree * 'a forest;;
    
type 'a mtree_lst = MTree of 'a * ('a mtree_lst) list;;

type 'a mtree = MNode of 'a * 'a forest
and 'a forest = EmptyForest | Forest of 'a mtree * 'a forest


type 'a mtree_lst = MTree of 'a * 'a mtree_lst list


In [15]:
let rec preorder_MTree = function
    | MTree(v,l) -> v :: List.fold_left (fun acc t -> acc @ preorder_MTree t) [] l

let rec fold_map_forest f = function
    | EmptyForest -> []
    | Forest(t,f') -> f t @ fold_map_forest f f'

let rec preorder_MNode = function
    | MNode(v,f) -> v :: fold_map_forest preorder_MNode f

let bfs_MTree t =  
    let rec aux = function
        | [] -> []
        | MTree(v,l) :: ls -> v :: aux (ls @ l) 
    in
        aux [t]

let bfs_MNode t =
    let rec aux = function
        | [] -> []
        | MNode(v,f) :: ls -> v :: aux (ls @ fold_map_forest (fun a -> [a]) f)
    in
        aux [t]

val preorder_MTree : 'a mtree_lst -> 'a list = <fun>


val fold_map_forest : ('a mtree -> 'b list) -> 'a forest -> 'b list = <fun>


val preorder_MNode : 'a mtree -> 'a list = <fun>


val bfs_MTree : 'a mtree_lst -> 'a list = <fun>


val bfs_MNode : 'a mtree -> 'a list = <fun>


In [16]:
let test_MTree = MTree(1,[  MTree(2,[ MTree(3,[]) ; MTree(4,[])   ]) ; MTree(5,[]) ;MTree(6,[]) ]);;
let test_MNode = 
MNode(1,Forest(MNode(2, Forest(MNode(3,EmptyForest),Forest(MNode(4,EmptyForest),EmptyForest))), Forest(MNode(5,EmptyForest), Forest(MNode(6,EmptyForest),EmptyForest) )  ));;

preorder_MTree test_MTree;;
preorder_MNode test_MNode;;
bfs_MTree test_MTree;;
bfs_MNode test_MNode;;

val test_MTree : int mtree_lst =
  MTree (1,
   [MTree (2, [MTree (3, []); MTree (4, [])]); MTree (5, []); MTree (6, [])])


val test_MNode : int mtree =
  MNode (1,
   Forest
    (MNode (2,
      Forest (MNode (3, EmptyForest),
       Forest (MNode (4, EmptyForest), EmptyForest))),
    Forest (MNode (5, EmptyForest),
     Forest (MNode (6, EmptyForest), EmptyForest))))


- : int list = [1; 2; 3; 4; 5; 6]


- : int list = [1; 2; 3; 4; 5; 6]


- : int list = [1; 2; 5; 6; 3; 4]


- : int list = [1; 2; 5; 6; 3; 4]


## Zadanie 4 (6 pkt.)

Zdefiniuj typ służący do reprezentacji formuł rachunku zdań składających się ze zmiennych zdaniowych, negacji, koniunkcji i alternatywy.

1. Napisz funkcję sprawdzającą, czy dana formuła jest tautologią. W tym celu należy generować kolejne wartościowania zmiennych zdaniowych występujących w danej formule i sprawdzać dla nich wartość formuły. W przypadku, gdy formuła nie jest tautologią, funkcja powinna, oprócz tej informacji, podać jedno z wartościowań, dla których formuła nie jest prawdziwa.
2. Napisz funkcję, która przekształca zadaną formułę w formułę jej równoważną w negacyjnej postaci normalnej, tj. w której negacja występuje tylko przy zmiennych zdaniowych.
3. Napisz funkcję, która przekształca zadaną formułę w formułę jej równoważną w koniunkcyjnej postaci normalnej.
4. Napisz funkcję sprawdzającą (syntaktycznie), czy zadana formuła jest tautologią, korzystając z faktu, że każdą formułę można przedstawić w koniunkcyjnej postaci normalnej.
5. Napisz funkcję, która przekształca zadaną formułę w formułę jej równoważną w dyzjunkcyjnej postaci normalnej.
6. Napisz funkcję sprawdzającą (syntaktycznie), czy zadana formuła jest sprzeczna, korzystając z faktu, że każdą formułę można przedstawić w dyzjunkcyjnej postaci normalnej.

In [25]:
type 'a formula =    
    | Var of 'a 
    | And of 'a formula * 'a formula 
    | Or of 'a formula * 'a formula 
    | Not of 'a formula

type 'a formula =
    Var of 'a
  | And of 'a formula * 'a formula
  | Or of 'a formula * 'a formula
  | Not of 'a formula


In [57]:
let formula1 p = Var(p);;
let formula2 p = Not(Var(p));;
let tautology1 p = Or(Var(p),Not(Var(p)))
let tautology2 p q = Or(tautology1 p, Var(q))

let rec eval_formula = function
    | Var(true) -> true
    | Var(false) -> false
    | Not(p) -> not (eval_formula p)
    | And(p,q) -> eval_formula p && eval_formula q
    | Or(p,q) -> eval_formula p || eval_formula q



val formula1 : 'a -> 'a formula = <fun>


val formula2 : 'a -> 'a formula = <fun>


val tautology1 : 'a -> 'a formula = <fun>


val tautology2 : 'a -> 'a -> 'a formula = <fun>


val eval_formula : bool formula -> bool = <fun>


In [56]:
eval_formula @@ tautology2 false 

error: error

## Zadanie 5 (3 pkt.)

Funkcja jest napisana w stylu przekazywania kontynuacji (continuation-passing style, CPS), jeżeli przyjmuje dodatkowy argument funkcyjny zwany kontynuacją, który reprezentuje całą resztę obliczeń jakie mają zostać przeprowadzone po powrocie z tej funkcji. W konsekwencji, funkcje w CPS-ie zwracają wynik wywołując swoją kontynuację, a wszystkie wywołania w programie w CPS-ie są ogonowe. Na przykład, funkcja licząca silnię napisana w CPS-ie wygląda następująco:

`let rec fact_cps n k = if n = 0 then k 1 else fact_cps (n-1) (fun v -> k (n*v))`

Kontynuacja początkowa przekazana funkcji factcps mówi co zrobić z wynikiem obliczenia silni zadanej liczby. Typowe wywołanie funkcji factcps dostaje identyczność jako kontynuację początkową (gdy silnia jest obliczona, wystarczy ją zwrócić):

`let fact n = fact_cps n (fun v -> v)`

Dla drzew binarnych z zadania 2, napisz funkcję `prod : int btree -> int`, która liczy iloczyn wszystkich wartości w drzewie. Zapisz tę funkcję w CPS-ie, a następnie zmodyfikuj otrzymaną funkcję tak by w przypadku napotkania wartości 0 funkcja wykonała bezpośredni skok do miejsca swego wywołania, bez krokowego powracania z rekursji.

## Zadanie 6 (5 pkt.)

Gdy modyfikujemy strukturę danych, często chcemy wykonać więcej niż jedną operację w pobliskich miejscach. W przypadku czysto funkcyjnych struktur może to skutkować niepotrzebnie długim czasem wykonania, spowodowanym przechodzeniem i odtwarzaniem struktury danych przy każdej zmianie. Aby zapobiec temu problemowi, możemy rozbić operację modyfikacji struktury na dwie części: znajdowania miejsca w strukturze, w którym mamy wykonać zmianę, oraz przeprowadzenia zmiany w konkretnym miejscu; musimy też znaleźć odpowiednią pośrednią reprezentację dla takich miejsc.

1. Zdefiniuj typ danych α place jako pośrednią reprezentację dla typu α list.
2. Zaimplementuj funkcje findNth : α list -> int -> α place oraz collapse : α place -> α list. Pierwsza powinna lokalizować n-te miejsce na liście, druga — zapominać informację o miejscu i zwracać listę na której działamy.
3. Zaimplementuj funkcje add : α -> α place -> α place oraz del : α place -> α place, odpowiednio dodającą i usuwającą element w odpowiednim miejscu listy. Funkcje powinny działać w czasie stałym, oraz spełniać następujące równości:

`collapse (add 3 (findNth [1;2;4] 2)) = [1;2;3;4]
collapse (del (findNth [1;2;4] 2)) = [1;2]
del (add x p) = p dla dowolnych x : α i p : α place`

4. Zaimplementuj funkcje next: α place -> α place oraz prev : α place -> α place, służące do nawigacji w strukturze listy. Te funkcje również powinny działać w czasie stałym.
5. Listy nie są jedynym typem danych dla którego możemy definiować struktury pamiętające miejsce w którym pracujemy. Zdefiniuj typ danych α btplace będący pośrednią reprezentacją pozwalającą na modyfikacje drzew binarnych z Zadania 2 (nie musisz definiować na nim odpowiednich operacji).